# Predicting Sentiment From Product Reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!

# Import Neccessary Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.

In [4]:
products.iloc[269,:]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    text = text.translate(str.maketrans('','',string.punctuation)) 
    
    return text


products = products.fillna({'review':''})  # fill in N/A's in the review column
products['review_clean'] = products['review'].apply(remove_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [6]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [7]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
...,...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,Such a great idea very handy to have and look ...,1
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,This product rocks It is a great blend of fun...,1
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,This item looks great and cool for my kidsI kn...,1
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,I am extremely happy with this product I have ...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. However, this notebook uses the indexes from the respective json files to get the indexes for train and test datasets.

In [8]:
# Load the json files as lists consisting the indices for the train_data and the test_data
train_indices = pd.read_json('module-2-assignment-train-idx.json')
train_indices = train_indices.rename({0: 'indices'}, axis=1)
train_indices = list(train_indices['indices']) # This ensures that the indices are not stored as list of lists

test_indices = pd.read_json('module-2-assignment-test-idx.json')
test_indices = test_indices.rename({0: 'indices'}, axis=1)
test_indices = list(test_indices['indices']) # This ensures that the indices are not stored as list of lists

# Filter the train_data and test_data based on the train_indices and test_indices
train_data = products.iloc[train_indices]
test_data = products.iloc[test_indices]

print('train length :', len(train_data))
print('test length :', len(test_data))

train length : 133416
test length : 33336


## Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

* Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
* Compute the occurrences of the words in each review and collect them into a row vector.
* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
* Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

In [9]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix

train_matrix = vectorizer.fit_transform(train_data['review_clean'])

# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

Keep in mind that the test data must be transformed in the same way as the training data.

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix **`train_matrix` as features** and the **column sentiment of `train_data` as the target**. Use the default values for other parameters. Call this model sentiment_model.

**Note:** This line may take 1-2 minutes.

In [10]:
clf = LogisticRegression()
sentiment_model = clf.fit(train_matrix, train_data['sentiment'])

print('Classifier Details : ',clf, '\n')
print('Sentiment Model Coefficients :\n',sentiment_model.coef_,'\n')
print('Number of Sentiment Model Coefficients : ', sentiment_model.coef_.size)

# The code below is to answer the following quiz question
non_zero_coefs = (sentiment_model.coef_>= 0).sum()
total_num_coefs = np.size(sentiment_model.coef_) # used in Week 3 Quiz
print(non_zero_coefs, 'coefficients have weights >=0')

Classifier Details :  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 

Sentiment Model Coefficients :
 [[-6.28825429e-01  1.10286789e-03  8.64651593e-03 ...  1.49255140e-03
   1.07351416e-03 -4.16512629e-04]] 

Number of Sentiment Model Coefficients :  121712
90353 coefficients have weights >=0


c:\users\ahmed ismail khalid\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

<font color = 'steelblue'><b> Quiz : How many weights are >= 0? </b></font>

<font color = 'mediumvioletred'><b> Answer : {{non_zero_coefs}} coefficients have weights >=0 </b></font>

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [11]:
sample_test_data = test_data[10:13]
sample_test_data

,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the sample_test_data. Here's the full review:

In [12]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the sample_test_data looks like. As we could guess from the rating (-1), the review is quite negative.

In [13]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [14]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
scores

array([  4.85336487,  -3.16175571, -10.11461459])

### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions for **sample_test_data**

**Checkpoint:** Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

In [15]:
# Calculating predictions
predictions = [1 if score > 0 else -1 for score in scores]

# Checkpoint : Making sure predictions match the ones obtained from sentiment_model
predictions_checkpoint = clf.predict(sample_test_matrix)

print('Predictions from code            : ', predictions)
print('Predictions from sentiment_model : ', predictions)

Predictions from code            :  [1, -1, -1]
Predictions from sentiment_model :  [1, -1, -1]


### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

**Checkpoint:** Make sure your probability predictions match the ones obtained from sentiment_model.

In [16]:
# Calculating probabilty predictions
probability_predictions = 1 / (1 + np.exp(-scores))

# Checkpoint : Making sure probabilty predictions match the ones obtained from sentiment_model
probability_predictions_checkpoint = clf.predict_proba(sample_test_matrix)

print('Predictions from code            : ', probability_predictions)
print('Predictions from sentiment_model : \n', probability_predictions_checkpoint)


# The code below is to answer the following quiz question 

lowest = []
for i in range(len(probability_predictions_checkpoint)) :
    lowest.append(probability_predictions_checkpoint[i][1])
    
model_names = ['First', 'Second', 'Third']

lowest_dict = dict(zip(model_names, lowest))
lowest_index = min(lowest_dict, key=lowest_dict.get)

print('\n',lowest_dict,'\n\n')
print(lowest_index, 'model has the lowest probability of being classified as positive review')

Predictions from code            :  [9.92258321e-01 4.06305612e-02 4.04819213e-05]
Predictions from sentiment_model : 
 [[7.74167918e-03 9.92258321e-01]
 [9.59369439e-01 4.06305612e-02]
 [9.99959518e-01 4.04819213e-05]]

 {'First': 0.9922583208242403, 'Second': 0.040630561175331814, 'Third': 4.048192132535376e-05} 


Third model has the lowest probability of being classified as positive review


<font color = 'steelblue'><b> Quiz : Of the three data points in `sample_test_data`, which one (first, second, or third) has the *lowest probability* of being classified as a positive review? </b></font>

<font color = 'mediumvioletred'><b> Answer : {{lowest_index}} model has the lowest probability of being classified as positive review </b></font>

# Find the most positive (and negative) review

We now turn to examining the full test dataset, test_data, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the **"most positive reviews."**

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`.
2.  Sort the data according to those predictions and pick the top 20.

In [17]:
test_matrix = vectorizer.transform(test_data['review_clean'])
scores_test = sentiment_model.decision_function(test_matrix)
predictions = 1/(1+np.exp(-scores_test))


test_data['predictions'] = predictions
pos_test_data = test_data.sort_values(by='predictions',ascending=False).iloc[0:20]
most_positive_reviews = pos_test_data['review']

# The code below is to answer the following quiz question
positive_items = pos_test_data['name'].unique().tolist()

print('Items represented in the 20 most positive reviews are : \n')
for product in positive_items :
    print(product)
    
pos_test_data

print(len(positive_items))

Items represented in the 20 most positive reviews are : 

Roan Rocco Classic Pram Stroller 2-in-1 with Bassinet and Seat Unit - Coffee
Graco FastAction Fold Jogger Click Connect Stroller, Grapeade
Infantino Wrap and Tie Baby Carrier, Black Blueberries
Baby Einstein Around The World Discovery Center
Britax 2012 B-Agile Stroller, Red
Graco Pack 'n Play Element Playard - Flint
Evenflo X Sport Plus Convenience Stroller - Christina
Diono RadianRXT Convertible Car Seat, Plum
Fisher-Price Cradle 'N Swing,  My Little Snugabunny
Mamas &amp; Papas 2014 Urbo2 Stroller - Black
Stork Craft Beatrice Combo Tower Chest, White
Buttons Cloth Diaper Cover - One Size - 8 Color Options
Britax Boulevard 70-G3 Convertible Car Seat Seat, Onyx
Baby Jogger City Mini GT Single Stroller, Shadow/Orange
P'Kolino Silly Soft Seating in Tias, Green
Baby Jogger City Mini GT Double Stroller, Shadow/Orange
Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
Simple Wishes Hands-Free Breastpump Bra, Pink, XS-L
Ikea 36 Pcs Kalas K

c:\users\ahmed ismail khalid\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<font color = 'steelblue'><b> Quiz : Which of the following products are represented in the 20 most positive reviews? </b></font>

<font color = 'mediumvioletred'><b>  Answer : The products represented as the 20 most positive reviews are :  </b></font> 
<font color = 'lightslategray'><b> <p>{{positive_items}}</p> </b></font>

Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [18]:
neg_test_data = test_data.sort_values(by='predictions',ascending=True).iloc[0:20]

most_negative_reviews = neg_test_data['review']

# The code below is to answer the following quiz question
negative_items = neg_test_data['name'].unique().tolist()

print('Items represented in the 20 most negative reviews are : \n')
for product in negative_items :
    print(product)
    
neg_test_data

Items represented in the 20 most negative reviews are : 

The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit
Fisher-Price Ocean Wonders Aquarium Bouncer
Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)
VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor
Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)
Safety 1st High-Def Digital Monitor
One Step Ahead Hide-Away Extra Long Bed Rail
Cloth Diaper Sprayer--styles may vary
Baby Trend Inertia Infant Car Seat - Horizon
Snuza Portable Baby Movement Monitor
Samsung SEW-3037W Wireless Pan Tilt Video Baby Monitor Infrared Night Vision and Zoom, 3.5 inch
Regalo My Cot Portable Bed, Royal Blue
Ellaroo Mei Tai Baby Carrier - Hershey
Peg-Perego Tatamia High Chair, White Latte
Baby Jogger Summit XC Double Stroller, Red/Black
Safety 1st Exchangeable Tip 3 in 1 Thermometer
Safety 1st Deluxe 4-in-1 Bath Station
Motorola Digi

,name,review,rating,review_clean,sentiment,predictions
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,1.707690e-14
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,2.529871e-14
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,2.683606e-12
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,6.560807e-12
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,2.547734e-10
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,3.872547e-10
95420,One Step Ahead Hide-Away Extra Long Bed Rail,"I bought a brand new 56"" hide-away bed safety ...",1,I bought a brand new 56 hideaway bed safety ra...,-1,4.126846e-10
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,8.058510e-10
176046,Baby Trend Inertia Infant Car Seat - Horizon,"I really wanted to love this seat; however, I ...",1,I really wanted to love this seat however I wo...,-1,9.035887e-10
94389,Snuza Portable Baby Movement Monitor,I would have given the product 4 stars for whi...,1,I would have given the product 4 stars for whi...,-1,1.284190e-09


<font color = 'steelblue'><b> Quiz : Which of the following products are represented in the 20 most negative reviews? </b></font>

<font color = 'mediumvioletred'><b>  Answer : The products represented as the 20 most negative reviews are :  </b></font> 
<font color = 'lightslategray'><b> <p>{{negative_items}}</p> </b></font>

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [19]:
test_matrix_sentiment_model = vectorizer.transform(test_data['review_clean'])
scores_test_sentiment_model = sentiment_model.decision_function(test_matrix_sentiment_model)
predictions_test_sentiment_model = clf.predict(test_matrix_sentiment_model)

correct_test = sum([1 if prediction == true_label else 0 for prediction, true_label
                   in zip(predictions_test_sentiment_model, test_data['sentiment'])])

sentiment_model_accuracy_test = round(correct_test/len(test_data['review_clean']),2)

print('Sentiment Model Accuracy on Test Data : ', sentiment_model_accuracy_test)

data_test_sentiment_model = {'predictions':predictions_test_sentiment_model, 'original':test_data['sentiment']}
df_test_sentiment_model = pd.DataFrame(data=data_test_sentiment_model)

Sentiment Model Accuracy on Test Data :  0.93


<font color = 'steelblue'><b> Quiz 1: What is the accuracy of the `sentiment_model` on the `test_data`? Round your answer to 2 decimal places (e.g. 0.76) </b></font>

<font color = 'mediumvioletred'><b>  Answer 1 : Sentiment Model has an accuracy of {{sentiment_model_accuracy_test*100}}%  </b></font>

<br/>

<font color = 'steelblue'><b> Quiz 2: Does a higher accuracy value on the `training_data` always imply that the classifier is better? </b></font>

<font color = 'mediumvioletred'><b>  Answer 2 : No, as the this could imply overfitting on the training data as compared to generalizing on the data </b></font>

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [20]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [21]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Compute word count vectors for the training and test data and obtain the sparse matrices train_matrix_word_subset and test_matrix_word_subset, respectively.

## Train a logistic regression model on a subset of data

Now build a logistic regression classifier with **train_matrix_word_subset** as features and **sentiment** as the target. Call this model **simple_model**.

In [22]:
clf_simple = LogisticRegression()
simple_model = clf_simple.fit(train_matrix_word_subset, train_data['sentiment'])

Let us inspect the weights (coefficients) of the simple_model. First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running

In [23]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})
simple_model_coef_table

,word,coefficient
0,love,1.363697
1,great,0.943950
2,easy,1.192219
3,old,0.085424
4,little,0.520174
5,perfect,1.510263
6,loves,1.673269
7,well,0.503760
8,able,0.190937
9,car,0.058813


Sort the data frame by the coefficient value in descending order.

Note: Make sure that the intercept term is excluded from this table.

In [24]:
simple_model_coef_table = simple_model_coef_table.sort_values(by='coefficient', ascending=False)
simple_model_coef_table


# The code below is to answer the following quiz questions

count_positive = sum(simple_model_coef_table['coefficient'] > 0)
print(count_positive, 'of the 20 coefficients in the simple_model_coef_table are positive for simple_model')

positive_simple_model = simple_model_coef_table['word'][simple_model_coef_table['coefficient'] >= 0].tolist()

vocab = list(vectorizer.vocabulary_.keys())
coeffs = {vocab[i]: c for i, c in enumerate(sentiment_model.coef_[0])}

common_dict = {k:v for k, v in coeffs.items() if k in significant_words}

common_words = pd.DataFrame(common_dict.items(), columns=['word', 'simple_model coef'])
common_words = pd.merge(simple_model_coef_table, common_words, how = 'left', on = 'word')
common_words = common_words[common_words['coefficient'] >= 0]
common_words = common_words['word'].tolist()

print('\nWords that are present in both simple_model and sentiment_model are :', common_words)

10 of the 20 coefficients in the simple_model_coef_table are positive for simple_model

Words that are present in both simple_model and sentiment_model are : ['loves', 'perfect', 'love', 'easy', 'great', 'little', 'well', 'able', 'old', 'car']


<font color = 'steelblue'><b> Quiz 1 : Consider the coefficients of simple_model. How many of the 20 coefficients (corresponding to the 20 significant_words) are positive for the simple_model? </b></font>

<font color = 'mediumvioletred'><b>  Answer 1 : {{count_positive}} of the 20 coefficients in the simple_model_coef_table are positive for simple_model. These words are : </b></font>
<font color = 'slategray'><b> <p>{{positive_simple_model}}</p> </b></font>

<br/>

<font color = 'steelblue'><b> Quiz 2 : Are the positive words in the simple_model also positive words in the sentiment_model? </b></font>

<font color = 'mediumvioletred'><b>  Answer 2 : All the words present in the simple_model are also present in the sentiment_model. </b></font>

# Comparing models

We will now compare the accuracy of the sentiment_model and the simple_model.

First, compute the classification accuracy of the sentiment_model on the train_data.

In [25]:
predictions_train_sentiment_model = sentiment_model.predict(train_matrix)

correct_train_sentiment_model = sum([1 if prediction == true_label else 0 for prediction, true_label
                   in zip(predictions_train_sentiment_model, train_data['sentiment'])])

sentiment_model_accuracy_train = round(correct_train_sentiment_model/len(train_data['review_clean']),2)

print('Sentiment Model Accuracy on Train Data : ',sentiment_model_accuracy_train)

Sentiment Model Accuracy on Train Data :  0.95


Now, compute the classification accuracy of the simple_model on the train_data.

In [26]:
predictions_train_simple_model = simple_model.predict(train_matrix_word_subset)

correct_train_simple_model = sum([1 if prediction == true_label else 0 for prediction, true_label
                   in zip(predictions_train_simple_model, train_data['sentiment'])])

simple_model_accuracy_train = round(correct_train_simple_model/len(train_data['review_clean']),2)

print('Simple Model Accuracy on Train Data : ',simple_model_accuracy_train,'\n')


# The code below is to answer the following quiz question
if sentiment_model_accuracy_train > simple_model_accuracy_train :
    accurate_train_model = 'Sentiment Model'
    higher_accuracy_train = sentiment_model_accuracy_train
    
elif sentiment_model_accuracy_train < simple_model_accuracy_train :
    accurate_train_model = 'Simple Model'
    higher_accuracy_train = simple_model_accuracy_train
    
print(accurate_train_model, 'has the higher accuracy on TRAINING set with an accuracy of', higher_accuracy_train)

Simple Model Accuracy on Train Data :  0.87 

Sentiment Model has the higher accuracy on TRAINING set with an accuracy of 0.95


<font color = 'steelblue'><b> Quiz : Which model (sentiment_model or simple_model) has higher accuracy on the TRAINING set? </b></font>

<font color = 'mediumvioletred'><b>  Answer : {{accurate_train_model}} has the higher accuracy on TRAINING set with an accuracy of {{higher_accuracy_train}} </b></font>

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [27]:
predictions_test_sentiment_model = sentiment_model.predict(test_matrix)

correct_test_sentiment_model = sum([1 if prediction == true_label else 0 for prediction, true_label
                   in zip(predictions_test_sentiment_model, test_data['sentiment'])])

sentiment_model_accuracy_test = round(correct_test_sentiment_model/len(test_data['review_clean']),2)

print('Sentiment Model Accuracy on test Data : ',sentiment_model_accuracy_test)

Sentiment Model Accuracy on test Data :  0.93


Next, compute the classification accuracy of the simple_model on the test_data.

In [28]:
predictions_test_simple_model = simple_model.predict(test_matrix_word_subset)

correct_test_simple_model = sum([1 if prediction == true_label else 0 for prediction, true_label
                   in zip(predictions_test_simple_model, test_data['sentiment'])])

simple_model_accuracy_test = round(correct_test_simple_model/len(test_data['review_clean']),2)

print('Simple Model Accuracy on test Data : ',simple_model_accuracy_test,'\n')


# The code below is to answer the following quiz question
if sentiment_model_accuracy_test > simple_model_accuracy_test :
    accurate_test_model = 'Sentiment Model'
    higher_accuracy_test = sentiment_model_accuracy_test
    
elif sentiment_model_accuracy_test < simple_model_accuracy_test :
    accurate_test_model = 'Simple Model'
    higher_accuracy_test = simple_model_accuracy_test
    
print(accurate_test_model, 'has the higher accuracy on TEST set with an accuracy of', higher_accuracy_test)

Simple Model Accuracy on test Data :  0.87 

Sentiment Model has the higher accuracy on TEST set with an accuracy of 0.93


<font color = 'steelblue'><b> Quiz : Which model (sentiment_model or simple_model) has higher accuracy on the TEST set? </b></font>

<font color = 'mediumvioletred'><b>  Answer : {{accurate_test_model}} has the higher accuracy on TEST set with an accuracy of {{higher_accuracy_test}} </b></font>

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [29]:
class_pos_train = sum(train_data['sentiment'] == 1)
class_neg_train = sum(train_data['sentiment'] == -1)

if class_pos_train > class_neg_train :
    majority_train_class = 'Positive Sentiment Class'

elif class_pos_train < class_neg_train :
    majority_train_class = 'Negative Sentiment Class'
    
print(majority_train_class ,'is the majority class in train set')

# The code below is to answer the following quiz questions 
majority_classifier_test_acc = round(max(class_pos_train,class_neg_train) / (class_pos_train + class_neg_train), 2)

print('\nThe accuracy of majority classifier on test data is :', majority_classifier_test_acc, '\n')

if sentiment_model_accuracy_test > majority_classifier_test_acc :
    better_model = 'Sentiment Model is definitely better than Majority Classifier'

elif sentiment_model_accuracy_test < majority_classifier_test_acc :
    better_model = 'Sentiment Model is definitely not better than Majority Classifier'
    
print(better_model, 'as it has an accuracy of', sentiment_model_accuracy_test, 'compared to that of majority classifier' 
      ' having an accuarcy of', majority_classifier_test_acc)

Positive Sentiment Class is the majority class in train set

The accuracy of majority classifier on test data is : 0.84 

Sentiment Model is definitely better than Majority Classifier as it has an accuracy of 0.93 compared to that of majority classifier having an accuarcy of 0.84


<font color = 'steelblue'><b> Quiz 1 : Enter the accuracy of the majority class classifier model on the test_data. Round your answer to two decimal places (e.g. 0.76). </b></font>

<font color = 'mediumvioletred'><b>  Answer 1 : The accuracy of majority classifier on test data is {{majority_classifier_test_acc}} </b></font>

<br/>

<font color = 'steelblue'><b> Quiz 2 : Is the sentiment_model definitely better than the majority class classifier (the baseline)? </b></font>

<font color = 'mediumvioletred'><b>  Answer 2 : {{better_model}} </b></font>